# Purpose of notebook
This notebook examplifies the replication of the shapleyPermEx and shapleyPermRand functions from the R package 'sensitivity'. I implement the functions to see if we get the same results as the R package in order to consequently apply the methods to the respy model.

In [2]:
# import neccesary packages
import numpy as np
import openturns as ot
import pandas as pd
import chaospy as cp
import itertools

from econsa.sampling import cond_mvn

## Shapley function

Code of original R package:

- [sensitivity source: R/shapleyPermEx.R](https://rdrr.io/cran/sensitivity/src/R/shapleyPermEx.R)
- [sensitivity source: R/shapleyPermRand.R](https://rdrr.io/cran/sensitivity/src/R/shapleyPermRand.R)

In [42]:
# define shapley function as in the R package
def ShapleyPerm(method, m, model, Xall, Xcond, d, Nv, No, Ni=3):
    """
    """
    if (method == 'exact'):
        perms = list(itertools.permutations(range(d), d))
        perms = [list(i) for i in perms]
        m = len(perms)
    else:
        perms = np.zeros((m,d), dtype = np.int64)
        for i in range(m):
            perms[i] = np.random.permutation(d) # random permutation
    
    #------------------------------
    # Creation of the design matrix
    #------------------------------
    
    X = np.zeros((Nv+m*(d-1)*No*Ni, d)) 
    X[:Nv,:] = Xall(Nv)

    for p in range(m):
    
        pi = perms[p]
        pi_sorted = np.argsort(pi)
    
        for j in range(1,d):
        
            Sj = pi[:j] # set of the 0st-(j-1)th elements in pi      
            Sjc = pi[j:] # set of the jth-dth elements in pi
        
            xjcM = Xcond(No,Sjc,None,None)# sampled values of the inputs in Sjc

            for l in range(No):
                
                xjc = xjcM[l,:]
            
                # sample values of inputs in Sj conditional on xjc
                xj = Xcond(Ni, Sj, Sjc, xjc)
                xx = np.concatenate((xj, np.ones((Ni,1))*xjc), axis = 1)
                ind_inner = Nv + p*(d-1)*No*Ni + (j-1)*No*Ni + l*Ni
                X[ind_inner:(ind_inner + Ni),:] = xx[:,pi_sorted]
    
    #-----------------------
    # Calcul of the response
    #-----------------------
    
    y = model(X)

    #-----------------------------------------------------------------
    # Initialize Shapley, main and total Sobol effects for all players
    #-----------------------------------------------------------------
    
    Sh = np.zeros(d)
    Vsob = np.zeros(d)
    Tsob = np.zeros(d)
    
    nV = np.zeros(d) # number of samples used to estimate V1,...,Vd
    nT = np.zeros(d) # number of samples used to estimate T1,...,Td
    
    #----------------
    # Estimate Var[Y]
    #----------------
    
    Y = y[:Nv]
    y = y[Nv:]
    EY = np.mean(Y)
    VarY = np.var(Y)

    #-----------------------------------------------
    # Estimate Shapley, main and total Sobol effects
    #-----------------------------------------------
    
    cVar = np.zeros(No)

    for p in range(m):
    
        pi = perms[p]
        prevC = 0
    
        for j in range(d):
            if (j == (d-1)):
                Chat = VarY
                delta = Chat - prevC
                Vsob[pi[j]] = Vsob[pi[j]] + prevC # first order effect
                nV[pi[j]] = nV[pi[j]] + 1
            else:
                for l in range(No):
                    Y = y[:Ni]
                    y = y[Ni:]
                    cVar[l] = np.var(Y)
                Chat = np.mean(cVar)
                delta = Chat - prevC
      
            Sh[pi[j]] = Sh[pi[j]] + delta
        
            prevC = Chat
        
            if (j == 0):
                Tsob[pi[j]] = Tsob[pi[j]] + Chat # Total effect
                nT[pi[j]] = nT[pi[j]] + 1
    
    Sh = Sh / m / VarY
    
    if (method == 'exact'):
        Vsob = Vsob / (m/d) / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / (m/d) / VarY # averaging by number of permutations with j=1 
    else:
        Vsob = Vsob / nV / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / nT / VarY # averaging by number of permutations with j=1 
    
    col = ['X' + str(i) for i in np.arange(d)+1]
    effects = pd.DataFrame(np.array([Sh,Vsob,Tsob]), index = ['Shapley effects', 'First order Sobol', 'Total Sobol'], columns = col)

    return effects

In [4]:
def gaussian_model(X):
    return np.sum(X,1)

In [5]:
def Xall(n):
    distribution = ot.Normal(moyenne,cov)
    return distribution.getSample(n)

In [6]:
def Xcond(n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]
        cov_int = ot.CovarianceMatrix(cov_int)
        distribution = ot.Normal(moyenne[Sj],cov_int)
        return distribution.getSample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

# Conditional sampling functions
These funcitons estimate the conditional vector for use in the shapley function

In [8]:
# Generate conditional law
def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    cond_mean,cond_var = cond_mvn(mean, cov, dependent_ind = dependent_ind, given_ind = given_ind, given_value = X_given)
    
    cond_var = ot.CovarianceMatrix(cond_var)
    distribution = ot.Normal(cond_mean,cond_var)
    
    return distribution.getSample(n)

## Evaluation Shapley effects on linear test model model

In [43]:
# Ni = 3

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 3

index = ShapleyPerm(method, m, gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.189402  0.384875  0.425723
First order Sobol  0.427677  0.875271  0.914905
Total Sobol        0.071684  0.013446  0.051687




In [44]:
# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 3

index = ShapleyPerm(method, m, gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Random method 
                         X1        X2        X3
Shapley effects    0.180831  0.394950  0.424219
First order Sobol  0.403891  0.875058  0.915287
Total Sobol        0.070539  0.013090  0.053119



In [ ]:
# Ni = 100

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

# Exact method
method = 'exact'
m = None
Nv = 10**4
No = 10**3
Ni = 100

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

In [17]:
# Random method
method = 'random'
m = 6000
Nv = 10**4
No = 1
Ni = 100

index = ShapleyPerm(method,m,gaussian_model, Xall, Xcond, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.105989  0.417345  0.476666
First order Sobol  0.111941  0.818251  0.878008
Total Sobol        0.103202  0.019565  0.078287


Random method 
                         X1        X2        X3
Shapley effects    0.113581  0.424693  0.461726
First order Sobol  0.140519  0.823465  0.880937
Total Sobol        0.099871  0.019112  0.075670

